## 並行処理いろいろ

In [ ]:
"""
シングルスレッドで並行処理を実行する

イベントループ: 
- 全ての上位ループ
- この上でコルーチンが起動する

コルーチン:
- サブルーチンがエントリーからリターンまでを一つの処理単位とするのに対し、コルーチンはいったん処理を中断した後、続きから処理を再開できる。
- 要するに、「途中で中断可能な一連の処理の単位」である。

https://qiita.com/osorezugoing/items/d26921f0affd62b87858
"""

import asyncio

async def task(name):
    print(f'{name}() started')
    await asyncio.sleep(1)
    print(f'{name}() finished')

async def main():
    task1 = asyncio.create_task(task("task1"))  # タスクを作成
    task2 = asyncio.create_task(task("task2"))  # タスクを作成
    task3 = asyncio.create_task(task("task3"))  # タスクを作成
    await task1
    await task2
    await task3

# イベントループを起動し、その上でコルーチンを起動
# asyncio.run(main()) # NOTE: jupyter noteは、asyncioのイベントループ上で動作するので、イベントループを開始できない

# コルーチンを起動
await main()

In [ ]:
"""
threading.Threadによるマルチスレッド処理
"""

import threading

def task(name):
    print(f'{name} started')
    x = 0
    for i in range(1000000):
        x += i
    print(f'{name} finished:', x)

def main():
    thread1 = threading.Thread(target=task, args=("task1",))
    thread2 = threading.Thread(target=task, args=("task2",))
    thread1.start()
    thread2.start()
    # NOTE: join()を実行すると、この処理が終了するまで呼び出し元の処理をブロックできる。
    thread1.join()
    thread2.join()
    

main()

In [ ]:
"""
run_in_executorにより、別スレッドで処理を実行する
通常の関数をawaitできるようにする仕組み
"""

import asyncio
import concurrent.futures
import functools
from logging import getLogger
logger = getLogger(__name__)
MACOS = True

def task(name):
    print(f"{name} started ")
    s = sum(i for i in range(10 ** 7))
    print(f"{name} finished ")
    return s

async def main_with_thread_pool():
    loop = asyncio.get_running_loop()
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=5,    
    ) as pool:
        task1 = loop.run_in_executor(pool, functools.partial(task, name="task1"))
        task2 = loop.run_in_executor(pool, functools.partial(task, name="task2"))
        task3 = loop.run_in_executor(pool, functools.partial(task, name="task3"))
        task4 = loop.run_in_executor(pool, functools.partial(task, name="task4"))
        task5 = loop.run_in_executor(pool, functools.partial(task, name="task5"))
        result1 = await task1
        result2 = await task2
        result3 = await task3
        result4 = await task4
        result5 = await task5
        print('result:', result1, result2, result3, result4, result5)

async def main_with_process_pool():
    # NOTE: MACでは以下をjupyter上で動かせない
    # 参考: https://stackoverflow.com/questions/61860800/running-a-processpoolexecutor-in-ipython
    if MACOS is True:
        logger.error("This Code Can't Work On Jupyter Note On Mac OS")
        return
    loop = asyncio.get_running_loop()
    with concurrent.futures.ProcessPoolExecutor() as pool:
        task1 = loop.run_in_executor(pool, functools.partial(task, name="task1"))
        task2 = loop.run_in_executor(pool, functools.partial(task, name="task2"))
        result1 = await task1
        result2 = await task2
        print('result:', result1, result2)


if __name__ == '__main__':# これが必要
    # イベントループを起動し、その上でコルーチンを起動
    # asyncio.run(main())

    # コルーチンを起動
    await main_with_thread_pool()
    # await main_with_process_pool()


## ベンチマークしてみる


In [47]:
!python ./benchmark.py

normal
[0, 4949776, 9899795, 14849102, 19800910, 24749863, 29697797, 34649723, 39596788, 44551888, 49503076, 54449152, 59400316, 64351091, 69294567]
20.113468398000002
asyncio
[0, 4949700, 9900532, 14850046, 19800188, 24749657, 29700350, 34653381, 39600501, 44550277, 49499171, 54450559, 59399471, 64346995, 69300925]
19.99381468
PPE
[0, 4950091, 9900626, 14849909, 19800185, 24750298, 29700794, 34649726, 39601112, 44548547, 49498125, 54450835, 59396724, 64354916, 69296167]
10.171667543999995
TPE
[0, 4949834, 9899040, 14849089, 19799243, 24748007, 29700154, 34651810, 39598658, 44550197, 49500579, 54448893, 59402203, 64350542, 69298565]
20.117084968999997
